In [16]:
# imports
import numpy as np
#from numpy import array

In [41]:
##################################################################################################################
# This method takes the parameters of the simulation and calculates the truck's and trailers' corners from it.   #
# inputs:                              truck_translation    absolute translation of the truck                    #
#                                         truck_rotation    absolute rotation of the truck, value in degrees     #
#                                              CoR_truck    truck's CoR w.r.t. bottom left corner                #
#                                            truck_shape    shape of truck                                       #
#                                hitch_translation_truck    distance from truck's CoR to its hitch point         #
#                                 first_trailer_rotation    realitve angle of trailer1 w.r.t. truck              #
#                  hitch_translation_first_trailer_truck    distance from truck's hitch point to trailer1's CoR  #
#                                           CoR_trailer1    trailer1's CoR w.r.t. bottom left corner             #
#                                    first_trailer_shape    width and heigt of first trailer                     #
#         hitch_translation_first_trailer_second_trailer    distance from trailer1's CoR                         #
#                                                           to trailer2's hitch point                            #
#                                second_trailer_rotation    relative angle of trailer2 w.r.t. trailer1           #
#                                           CoR_trailer2    trailer2's CoR w.r.t. bottom left corner             #
#                                   second_trailer_shape    width and height of trailer2                         #
#                       hitch_translation_second_trailer    distance from trailer1's hitch point to its CoR      #
# outputs:                                       corners    [truck's trailer1's trailer2's] corners              #
##################################################################################################################


def get_corners(truck_translation, truck_rotation, CoR_truck, truck_shape, hitch_translation_truck,\
                first_trailer_rotation, hitch_translation_first_trailer_truck, CoR_trailer1, first_trailer_shape,\
                hitch_translation_first_trailer_second_trailer,\
                second_trailer_rotation, CoR_trailer2, second_trailer_shape, hitch_translation_second_trailer):
    
    #### corners of the truck CHECKED
    # calculate corners of the truck w.r.t. its CoR
    corners_truck_unrotated_atOrigin = np.array([[             0,              0],\
                                                 [truck_shape[0],              0],\
                                                 [truck_shape[0], truck_shape[1]],\
                                                 [             0, truck_shape[1]]]) - CoR_truck
    
    #corners_truck_rotated = np.empty([4, 2])
    corners_truck_rotated = np.empty([5, 2])
    # rotate the corners around truck's CoR
    for i in range(4):
        corners_truck_rotated[i, :] = vector_rotation(corners_truck_unrotated_atOrigin[i, :], truck_rotation)
        
    # translate them to where the truck's CoR is supposed to be
    corners_truck = truck_translation + corners_truck_rotated
    
    # coordinates of the truck's hitch
    hitch_truck = truck_translation + vector_rotation([hitch_translation_truck, 0], truck_rotation)
    corners_truck[4, :] = hitch_truck
    
    
    ##############################################################################################################
    #### corners of trailer 1 CHECKED
    # claculate corners of trailer 1 with CoR at global origin
    corners_trailer1_unrotated_atOrigin = np.array([[                     0,                      0],\
                                                    [first_trailer_shape[0],                      0],\
                                                    [first_trailer_shape[0], first_trailer_shape[1]],\
                                                    [                     0 ,first_trailer_shape[1]]]) - CoR_trailer1
    
    # move the origin to the end of the hitch
    #corners_trailer1_rotated = np.empty([4, 2])
    corners_trailer1_rotated = np.empty([5, 2])
    # rotate trailer 1
    for i in range(4):
        corners_trailer1_rotated[i, :] = vector_rotation(corners_trailer1_unrotated_atOrigin[i, :], truck_rotation + first_trailer_rotation)
    
    # translate so that trailer1's hitch is attached to truck at truck't hitch point
    corners_trailer1 = hitch_truck + corners_trailer1_rotated\
                     + vector_rotation(np.array([-hitch_translation_first_trailer_truck, 0]), truck_rotation + first_trailer_rotation)
    
    # coordinates of trailer1's hitch
    hitch_trailer1 = hitch_truck + vector_rotation([hitch_translation_first_trailer_second_trailer - hitch_translation_first_trailer_truck, 0], truck_rotation + first_trailer_rotation)
    corners_trailer1[4, :] = hitch_trailer1
    
    
    ##############################################################################################################
    #### corners of trailer 2
    # claculate corners of trailer 1 as if its bottom left corner were at the global origin
    corners_trailer2_unrotated_atOrigin = np.array([[                      0,                       0],\
                                                    [second_trailer_shape[0],                       0],\
                                                    [second_trailer_shape[0], second_trailer_shape[1]],\
                                                    [                      0, second_trailer_shape[1]]]) - CoR_trailer2
    
    # move the origin to the end of the hitch
    corners_trailer2_rotated = np.empty([5, 2])
    # rotate trailer 2
    for i in range(4):
        corners_trailer2_rotated[i, :] = vector_rotation(corners_trailer2_unrotated_atOrigin[i, :], truck_rotation + first_trailer_rotation + second_trailer_rotation)
    
    
    
    
    
    # translate them to where trailer 2 is attached to trailer 1
    corners_trailer2 = hitch_trailer1 + corners_trailer2_rotated\
                     + vector_rotation([-hitch_translation_second_trailer, 0], truck_rotation + first_trailer_rotation + second_trailer_rotation)
    corners_trailer2[4, :] = [0, 0]
    
    corners = np.concatenate((corners_truck, corners_trailer1, corners_trailer2), axis=1)
    
    return corners

In [24]:
import numpy as np
from numpy import array

def vector_rotation(vector,angle):
    rotation_matrix = array([[np.cos(np.deg2rad(angle)),-np.sin(np.deg2rad(angle))],
                             [np.sin(np.deg2rad(angle)),np.cos(np.deg2rad(angle))]])
    vector = np.dot(rotation_matrix,vector)
    return vector